## Class 2

Today we are going to go over a few useful tidbits, and open up to any questions. 

## Lists vs Arrays

As all or most of you have used python before, you probably know that lists differ from arrays (lists are a native python datatype, arrays are part of the numpy package). In certain instances, depending on the type of data you are working with, lists and arrays can be used interchangeably. However, there are situations when using lists over arrays or arrays over lists becomes very important. 

In [1]:
list1 = [1,2,3,4,5,5,6,5]
print list1*2

[1, 2, 3, 4, 5, 5, 6, 5, 1, 2, 3, 4, 5, 5, 6, 5]


In [2]:
import numpy as np
arr1 = np.array(list1)
print 2*arr1

[ 2  4  6  8 10 10 12 10]


As you can see, the behavior of lists and arrays are different here- array multiplication happens element-wise, that is, each value in list1 is multiplied by 2 (or by every value and another equal length array). On the other hand, lists have a behavior for multiplication that works like concatenation- here the elements of the list are simply repeated twice. When working with a lot of astronomical data (like fits images), we want to divide out by flat-fields or subtract darks- this is trivially done with arrays and much more difficult with lists.

There are some other differences to make note of: 

Numpy arrays are faster and take up less memory, so for large datasets arrays are usually the way to go. Summing a 10000 element list is 10x slower than summing the equivalent array. However, sometimes arrays are slower but a more convenient way of storing data.

## The Where Function

Let's say we have a bunch of data, and we need to restrict it in some way. For the following examples, I will be using the SDSS catalog of galaxies. We will need to filter these arrays of data about stellar mass, metalicity, etc. 

In [3]:
import pyfits as pf
import time
sfrhdu = pf.open('gal_totsfr_dr7_v5_2.fits')[1]
sfr_full = sfrhdu.data
start_time = time.clock()
sfr = sfr_full[np.where(sfr_full['AVG'] > -99)[0]]
print time.clock() - start_time

0.266541


It took the numpy function 0.26 seconds to return a list of indices for everywhere in the datatable (stored actually as a dictionary) had a value of >-99 (which is a flag that indicates good/bad values). Let's see how long the same process takes using the more traditional for loop:

In [4]:
values = sfr_full['Avg']
indices = []
start_time = time.clock()
for i in range(len(values)):
    if values[i] > -99.0:
        indices.append(i)
print time.clock() - start_time

1.743933


It took the for-loop 1.6 seconds to complete the same task, which is around a factor of six slower. In this small example, you might not mind waiting the extra time, but as the data you are dealing with grows larger, or you impose more conditions, the computation time really speeds up. For example, consider the following. 

In [7]:
masshdu = pf.open('totlgm_dr7_v5_2b.fit')[1]
mass_full = masshdu.data
zhdu = pf.open('gal_fiboh_dr7_v5_2.fits')[1]
z_full = zhdu.data
start_time = time.clock()
restrictions = np.where((sfr_full['AVG'] > -99) & (mass_full['AVG'] != -1) & (z_full['AVG'] > -99.9))[0]
print time.clock() - start_time
sfr = np.array(sfr_full[restrictions])
mass =np.array(mass_full[restrictions])
z = np.array(z_full[restrictions])


0.035223


In [10]:
indexes = []
values2 =mass_full['Avg']
values3 = z_full['Avg']
start_time = time.clock()
for i in range(len(values)):
    if (values[i]>-99.0) and (values2[i] !=-1) and (values3[i] >-99.9):
        indexes.append(i)
print time.clock() - start_time

4.414704


This time, the difference is even more pronounced- the where function finished **126 times faster** than the for-loop. This leads to a general piece of advice in designing any program, which is to avoid looping over data if at all possible. Sometimes it's impossible to get around, but often times there are more effecient functions like np.where which can get you what you need much faster. And ESPECIALLY try to avoid double loops (nested for-loops), which will decimate your runtime.

# Histograms